In [1]:
import customtkinter as ctk
import tkinter.messagebox as mb
from tkinter import filedialog
import mysql.connector
import csv
import tkinter.ttk as ttk

# Center Window
def center_window(win, width, height):
    screen_width = win.winfo_screenwidth()
    screen_height = win.winfo_screenheight()
    x = int((screen_width / 2) - (width / 2))
    y = int((screen_height / 2) - (height / 2))
    win.geometry(f"{width}x{height}+{x}+{y}")

# Database Connection
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",        # Replace with your MySQL username
        password="sivas200424mcr104",    # Replace with your MySQL password
        database="placement_db"
    )

# Admin Login
def check_login():
    username = login_user.get()
    password = login_pass.get()
    with open("admin_credentials.txt", "r") as file:
        for line in file:
            u, p = line.strip().split(",")
            if username == u and password == p:
                login_window.destroy()
                open_main_window()
                return
    mb.showerror("Login Failed", "Invalid credentials")

# Get form data
def get_form_data():
    return (
        roll_entry.get(),
        name_entry.get(),
        phone_entry.get(),
        dept_menu.get(),
        company_entry.get(),
        package_entry.get(),
        year_entry.get()
    )

# Clear fields
def clear_fields():
    for entry in [roll_entry, name_entry, phone_entry, company_entry, package_entry, year_entry]:
        entry.delete(0, 'end')
    dept_menu.set("Select Department")

# Submit data
def submit_data():
    data = get_form_data()
    if "" in data or data[3] == "Select Department":
        mb.showerror("Input Error", "Please fill all fields.")
        return
    try:
        con = connect_db()
        cur = con.cursor()
        cur.execute("""
            INSERT INTO placements 
            (roll_no, student_name, phone_no, department, company_name, package, year) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, data)
        con.commit()
        con.close()
        mb.showinfo("Success", "Data added!")
        clear_fields()
        load_data()
    except mysql.connector.IntegrityError:
        mb.showerror("Error", "Roll No already exists.")
    except Exception as e:
        mb.showerror("DB Error", str(e))

# Load all data
def load_data():
    table.delete(*table.get_children())
    con = connect_db()
    cur = con.cursor()
    cur.execute("SELECT * FROM placements")
    for row in cur.fetchall():
        table.insert("", "end", values=row)
    con.close()

# Search
def search_data():
    keyword = search_entry.get()
    like_kw = f"%{keyword}%"
    con = connect_db()
    cur = con.cursor()
    cur.execute("""
        SELECT * FROM placements 
        WHERE roll_no LIKE %s OR company_name LIKE %s
    """, (like_kw, like_kw))
    rows = cur.fetchall()
    table.delete(*table.get_children())
    for row in rows:
        table.insert("", "end", values=row)
    con.close()

# Export to CSV
def export_to_csv():
    filename = filedialog.asksaveasfilename(defaultextension=".csv")
    if not filename:
        return
    con = connect_db()
    cur = con.cursor()
    cur.execute("SELECT * FROM placements")
    rows = cur.fetchall()
    con.close()
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Roll No", "Name", "Phone", "Department", "Company", "Package", "Year"])
        writer.writerows(rows)
    mb.showinfo("Success", "Data exported!")

# Fill form from table
def fill_form_from_table(event):
    selected = table.focus()
    if selected:
        data = table.item(selected, 'values')
        roll_entry.delete(0, 'end'); roll_entry.insert(0, data[0])
        name_entry.delete(0, 'end'); name_entry.insert(0, data[1])
        phone_entry.delete(0, 'end'); phone_entry.insert(0, data[2])
        dept_menu.set(data[3])
        company_entry.delete(0, 'end'); company_entry.insert(0, data[4])
        package_entry.delete(0, 'end'); package_entry.insert(0, data[5])
        year_entry.delete(0, 'end'); year_entry.insert(0, data[6])

# Edit data
def edit_data():
    selected = table.focus()
    if not selected:
        mb.showwarning("Select Row", "Please select a record to edit.")
        return
    original_roll = table.item(selected, 'values')[0]
    data = get_form_data()
    if original_roll != data[0]:
        mb.showerror("Edit Error", "You can't change the Roll Number.")
        return
    try:
        con = connect_db()
        cur = con.cursor()
        cur.execute("""
            UPDATE placements 
            SET student_name=%s, phone_no=%s, department=%s, company_name=%s, package=%s, year=%s 
            WHERE roll_no=%s
        """, (data[1], data[2], data[3], data[4], data[5], data[6], original_roll))
        con.commit()
        con.close()
        mb.showinfo("Success", "Data updated!")
        clear_fields()
        load_data()
    except Exception as e:
        mb.showerror("DB Error", str(e))

# Delete data
def delete_data():
    selected = table.focus()
    if not selected:
        mb.showwarning("Select Row", "Please select a record to delete.")
        return
    roll_no = table.item(selected, 'values')[0]
    if mb.askyesno("Confirm Delete", "Are you sure?"):
        con = connect_db()
        cur = con.cursor()
        cur.execute("DELETE FROM placements WHERE roll_no=%s", (roll_no,))
        con.commit()
        con.close()
        mb.showinfo("Deleted", "Record deleted.")
        load_data()

# Filter data
def filter_data():
    year = filter_year.get()
    min_pack = min_package.get()
    max_pack = max_package.get()
    query = "SELECT * FROM placements WHERE 1=1"
    values = []
    if year:
        query += " AND year=%s"
        values.append(year)
    if min_pack and max_pack:
        query += " AND package BETWEEN %s AND %s"
        values += [min_pack, max_pack]
    con = connect_db()
    cur = con.cursor()
    cur.execute(query, values)
    rows = cur.fetchall()
    table.delete(*table.get_children())
    for row in rows:
        table.insert("", "end", values=row)
    con.close()

# Main App Window


  

def open_main_window():
    global roll_entry, name_entry, phone_entry, dept_menu
    global company_entry, package_entry, year_entry
    global search_entry, filter_year, min_package, max_package, table

    app = ctk.CTk()
    app.title("Placement Tracker")
    center_window(app, 1100, 750)

    ctk.CTkLabel(app, text="Placement Tracker System", font=ctk.CTkFont(size=24, weight="bold")).pack(pady=10)

    # Wrapper frame for centering content
    content_frame = ctk.CTkFrame(app)
    content_frame.pack(pady=10, padx=10, fill="both", expand=True)

    # Inner frame for form
    frame = ctk.CTkFrame(content_frame)
    frame.pack(anchor="center", pady=10)

    labels = ["Roll No", "Name", "Phone", "Department", "Company", "Package", "Year"]
    roll_entry = ctk.CTkEntry(frame, width=200)
    name_entry = ctk.CTkEntry(frame, width=200)
    phone_entry = ctk.CTkEntry(frame, width=200)
    dept_menu = ctk.CTkOptionMenu(frame, width=200, values=[
        "MCA", "MBA", "MSC", "BE CSE", "B.TECH IT", "BE MECH", "BE ECE", "BE EEE"])
    company_entry = ctk.CTkEntry(frame, width=200)
    package_entry = ctk.CTkEntry(frame, width=200)
    year_entry = ctk.CTkEntry(frame, width=200)
    widgets = [roll_entry, name_entry, phone_entry, dept_menu, company_entry, package_entry, year_entry]

    for i, (label, widget) in enumerate(zip(labels, widgets)):
        ctk.CTkLabel(frame, text=label).grid(row=i, column=0, pady=5, sticky='e', padx=(0, 10))
        widget.grid(row=i, column=1, pady=5)

    ctk.CTkButton(frame, text="Submit", command=submit_data).grid(row=0, column=2, padx=10)
    ctk.CTkButton(frame, text="Edit", command=edit_data).grid(row=1, column=2, padx=10)
    ctk.CTkButton(frame, text="Delete", command=delete_data).grid(row=2, column=2, padx=10)
    ctk.CTkButton(frame, text="Export CSV", command=export_to_csv).grid(row=3, column=2, padx=10)
    ctk.CTkButton(frame, text="Clear", command=clear_fields).grid(row=4, column=2, padx=10)

    search_entry = ctk.CTkEntry(frame, placeholder_text="Search Roll No / Company", width=200)
    search_entry.grid(row=8, column=0, pady=10)
    ctk.CTkButton(frame, text="Search", command=search_data).grid(row=8, column=1)

    filter_year = ctk.CTkEntry(frame, placeholder_text="Year", width=100)
    min_package = ctk.CTkEntry(frame, placeholder_text="Min Package", width=100)
    max_package = ctk.CTkEntry(frame, placeholder_text="Max Package", width=100)
    filter_year.grid(row=9, column=0, pady=5)
    min_package.grid(row=9, column=1, pady=5)
    max_package.grid(row=9, column=2, pady=5)
    ctk.CTkButton(frame, text="Apply Filter", command=filter_data).grid(row=9, column=3, padx=10)

    table = ttk.Treeview(frame, columns=("Roll No", "Name", "Phone", "Department", "Company", "Package", "Year"), show="headings")
    for col in table["columns"]:
        table.heading(col, text=col)
        table.column(col, width=120)
    table.grid(row=10, column=0, columnspan=5, pady=20)
    table.bind("<ButtonRelease-1>", fill_form_from_table)

    load_data()
    app.mainloop()

# Login UI

login_window = ctk.CTk()
login_window.title("Admin Login")

# Increased window size
center_window(login_window, 600, 400)

# Larger font for title and inputs
title_font = ctk.CTkFont(size=28, weight="bold")
entry_font = ctk.CTkFont(size=16)

ctk.CTkLabel(login_window, text="Admin Login", font=title_font).pack(pady=20)

# Larger entry boxes
login_user = ctk.CTkEntry(login_window, placeholder_text="Username", font=entry_font, width=300, height=40)
login_user.pack(pady=15)

login_pass = ctk.CTkEntry(login_window, placeholder_text="Password", show="*", font=entry_font, width=300, height=40)
login_pass.pack(pady=15)

# Larger button
ctk.CTkButton(login_window, text="Login", command=check_login, width=150, height=40, font=entry_font).pack(pady=20)

login_window.mainloop()
